<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Tumoral Vessels - MSLD Segmentation
</p>
<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 1rem; '> 
    (using a single stitched image as input)
</p>

### Init

##### General Import

In [1]:
import numpy as np
import cv2
import os
from jppype import View2D, imshow, sync_views
from ipywidgets import GridspecLayout


def vlayout(*views, height='860px'):
    grid = GridspecLayout(len(views),1, height=height)
    sync_views(*views)
    for i, v in enumerate(views):
        grid[i,0] = v
    return grid
def hlayout(*views, height='600px'):
    grid = GridspecLayout(1,len(views), height=height)
    sync_views(*views)
    for i, v in enumerate(views):
        grid[0,i] = v
    return grid

In [2]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

##### Library Import

In [3]:
from microscopy_vessels_toolkit.preprocess.roi import *
from microscopy_vessels_toolkit.preprocess.enhance import *
from microscopy_vessels_toolkit.segment import segment
from microscopy_vessels_toolkit.analysis.vessel_density import *

### Load image

In [4]:
img, mask = load_img_mask("../DATA/1-images/G5_Hydrogel_only_31_select.png")
img_display = img.copy()
img_display[~mask] = 255 
v = imshow(img_display)
v

View2D()

## Find ROI ellipse

In [5]:
roi_yx, roi_normals = extract_tumor_ellipse(img, with_normals=True)

img_display[roi_yx[...,0], roi_yx[...,1]] = [255, 255, 255]
v['Image 01'] = img_display

## Find scale

In [6]:
scale, scale_mask = find_scale(img, mask, return_scale_mask=True)
scale_y, scale_x = np.where(scale_mask)
img_display[scale_y, scale_x] = [255,0,255]

v['Image 01'] = img_display
print(f"Detected scale: {scale:.2f}µm/px" )

Detected scale: 6.52µm/px


## Neighborhood Unwraping

Identify the region of the ellipse which face visible regions of the image (not masked out regions)

In [7]:
h = 1500/scale # 1mm
ROIs = extract_ROIs(roi_yx, roi_normals, mask, roi_h=h)

print(f"{len(ROIs)} regions were found.")
for roi in ROIs:
    print("\t*",str(roi))
    
roi_labels = np.zeros(img.shape[:2], dtype=np.uint8)
for i, roi in enumerate(ROIs):
    #roi.draw_polygon(roi_labels, color=i+1)
    roi_labels[roi.mask(roi_labels.shape)] = i+1
v.add_label(roi_labels, 'ROI label', options={'opacity': .3})
None

1 regions were found.
	* (854, 494) -> (1681, 821): 3359 px


In [8]:
ROIs = sorted(ROIs, key=lambda roi: roi.yx.shape[0], reverse=True)

In [9]:
raw_unwarped = [roi.unwarp(img) for roi in ROIs]
preprocessed = enhance_microscopy(img, mask, denoise=5)
pre_unwarped = [roi.unwarp(preprocessed) for roi in ROIs]
mask_unwarped = [roi.unwarp(mask) for roi in ROIs]

from IPython.display import display
for raw, pre in zip(raw_unwarped, pre_unwarped):
    display(vlayout(imshow(raw[::-1]), imshow(pre[::-1]), height='350px'))

GridspecLayout(children=(View2D(layout=Layout(grid_area='widget001'), linkedTransformGroup='f6c55691aa4549dfb7…

## Segment

In [10]:
raw = raw_unwarped[0][::-1]
pre = pre_unwarped[0][::-1]
mask = mask_unwarped[0][::-1]

from skimage.morphology import binary_closing
mask = binary_closing(mask, np.ones((5,5)))

In [11]:
vessels, vessels_refined = segment(pre, mask, refine_small_vessels=True, large_vessels=False, return_all_masks=True)

In [13]:
from jppype.utilities.color import colormap_by_name
cmap = colormap_by_name()

igc = 255-pre[...,1]
raw_display = raw.copy()
raw_display[~mask] = 255

pre_display = pre.copy()
pre_display[~mask] = 255

v_global = imshow(pre_display)
v_msld = imshow(igc)
v_msld.add_label(vessels_refined+2*vessels, 'vessels', colormap={1:cmap[0], 2:cmap[3],3:cmap[0]}, blend_mode='color')
v_lines = imshow(pre_display)
cmap = colormap_by_name()
v_lines.add_label(vessels_refined, blend_mode='color')
vlayout(v_msld, v_global,v_lines)

GridspecLayout(children=(View2D(layout=Layout(grid_area='widget001'), linkedTransformGroup='df3863d7f3f64275a7…

## Count Vessels

In [14]:
labels_view = np.zeros_like(vessels_refined, dtype=np.int32)

lines_labels = []
lines_row = np.linspace(0, vessels_refined.shape[0], 15 + 2, dtype=int)[1:-1]
for i in lines_row:
    i = int(round(i))
    line_label = label_1d(vessels_refined[i]!=0)
    lines_labels += [line_label]
    labels_view[i] = np.where(line_label!=0, line_label+labels_view.max(), 0)
    
v_lines.add_label(labels_view, f'VesselsLabel', opacity=1)
None

In [15]:
lines_nb_vessels = [line_label.max() for line_label in lines_labels]
nb_vessels_hist, edges = np.histogram(lines_nb_vessels)

p = figure(title='Vessels Count Distribution', tools='', background_fill_color="#fafafa")
p.quad(top=nb_vessels_hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)

p.y_range.start = 0
p.xaxis.axis_label = 'Vessel Count'
p.yaxis.axis_label = 'Number of Lines'
p.grid.grid_line_color="white"

show(p)

In [16]:
arg_median = np.argsort(lines_nb_vessels)[len(lines_nb_vessels)//2]
vr = np.bincount(lines_labels[arg_median].astype(np.int64))[1:]

maxr = vr.max()
v_hist, edges = np.histogram(vr, bins=maxr, range=[0,maxr])

p = figure(title='Median Vessels Size Distribution', tools='', background_fill_color="#fafafa")
p.quad(top=v_hist, bottom=0, left=edges[:-1]*3, right=edges[1:]*3,
       fill_color="navy", line_color="white", alpha=0.5)

p.y_range.start = 0
p.xaxis.axis_label = 'Size in um'
p.yaxis.axis_label = 'Number of Vessels'
p.grid.grid_line_color="white"

show(p)